In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import sklearn 
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

In [2]:
with open("./shakespeare.txt", mode="r") as f:
    text = f.read()

In [3]:
print(len(text))
print(text[:10])

1115394
First Citi


In [4]:
# 1.generate vocabulary
vocabulary = sorted(set(text))
print(len(vocabulary))
print(vocabulary)

65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [5]:
# 2.build mapping
char2idx = {value:key for key, value in enumerate(vocabulary)}
print(char2idx)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


In [6]:
# 根据索引找文字
idx2char = np.array(vocabulary)
print(idx2char)

['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']


In [7]:
# 3.transfer from char to index
text_as_index = [char2idx[c] for c in text]

print(len(text_as_index))
print(text_as_index[:10])
print(text[:10])

1115394
[18, 47, 56, 57, 58, 1, 15, 47, 58, 47]
First Citi


In [8]:
# from_tensor_slices: 数组 =》 dataset
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_index)

In [9]:
for i in char_dataset.take(2):
    print(i, idx2char[i])

tf.Tensor(18, shape=(), dtype=int32) F
tf.Tensor(47, shape=(), dtype=int32) i


In [10]:
# batch: 分成batch大小的dataset
# batch_size:表示要在单个批次中合并的此数据集的连续元素个数
# drop_remainder：表示在少于batch_size元素的情况下是否应删除最后一批

seq_length = 100
seq_dataset = char_dataset.batch(batch_size=seq_length + 1, drop_remainder=True)

In [11]:
# 1115394 / 101 = 11043
len(seq_dataset)

11043

In [12]:
# 查看数据集元素
for seq in seq_dataset.take(2):
    print(seq)
    # idx => char
    print(repr("".join([idx2char[index] for index in seq])))

tf.Tensor(
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1], shape=(101,), dtype=int32)
'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
tf.Tensor(
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1 49], shape=(101,), dtype=int32)
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'


In [13]:
# 4.预测文本 
# data: abcd, return: abc,bcd
def split_input_target(data):
    return data[0:-1], data[1:]

In [14]:
seq_dataset = seq_dataset.map(split_input_target)

# inputdata, outputdata: shape=100
for inputdata, outputdata in seq_dataset.take(2):
    print(inputdata)
    print(outputdata)

tf.Tensor(
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59], shape=(100,), dtype=int32)
tf.Tensor(
[47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43  1
 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43 39
 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49  6
  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0
 37 53 59  1], shape=(100,), dtype=int32)
tf.Tensor(
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 

In [15]:
batch_size = 64
buffer_size = 1000

seq_dataset = seq_dataset.shuffle(buffer_size).batch(
    batch_size, drop_remainder=True)

# 11043/64 = 172
print(len(seq_dataset))

172


### LSTM

In [16]:
def build_model(vocab_size, embedding_dim, batch_size, units):
    model = keras.models.Sequential([
        keras.layers.Embedding(
            input_dim = vocab_size, 
            output_dim = embedding_dim, 
            # [batch_size, None]:元素个数确定， 其他未定
            batch_input_shape = [batch_size, None]),
        keras.layers.LSTM(
            # units：输出空间的维度
            # stateful：默认 False，如果为 True，则批次中索引 i 处的每个样品的最后状态将用作下一批次中索引 i 样品的初始状态。
            # recurrent_initializer
            # return_sequences：是返回输出序列中的最后一个输出，还是全部序列
            units=units, 
            stateful = True,
            recurrent_initializer = 'glorot_uniform',
            return_sequences=True),
        keras.layers.Dense(vocab_size),
    ])
    
    return model

In [17]:
vocab_size = len(vocabulary)
embedding_dim = 256
units = 1024
batch_size = 64

model = build_model(vocab_size, embedding_dim, batch_size, units)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [18]:
# 对于每个字符，模型会查找嵌入，把嵌入当作输入运行一个时间步，并用密集层生成逻辑回归 （logits），预测下一个字符的对数可能性。
# 预测文本
for input_example_batch, target_example_batch in seq_dataset.take(1):
    print(input_example_batch, input_example_batch.shape)
    print(target_example_batch, target_example_batch.shape)
    # 函数式调用
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape)

tf.Tensor(
[[53 52  1 ... 56 47 52]
 [ 0 13 57 ... 25 63  1]
 [43 10  0 ... 58  1 58]
 ...
 [40 50 39 ... 39 56 58]
 [53 11  1 ... 46  1 46]
 [ 1 42 43 ...  1 55 59]], shape=(64, 100), dtype=int32) (64, 100)
tf.Tensor(
[[52  1 58 ... 47 52 45]
 [13 57  1 ... 63  1 52]
 [10  0 21 ...  1 58 56]
 ...
 [50 39 64 ... 56 58  1]
 [11  1 52 ...  1 46 47]
 [42 43 50 ... 55 59 53]], shape=(64, 100), dtype=int32) (64, 100)
(64, 100, 65)


In [19]:
# tf.random.categorical：从一个分类分布中抽取样本
# logits: 形状为 [batch_size, num_classes]的张量
# num_samples：从每一行切片中抽取的独立样本的数量
# [batch_size * num_classes] => [batch_size * num_samples] , [100, 65] => [100, 1]
sample_indices = tf.random.categorical(logits=example_batch_predictions[0], num_samples=1)
print(sample_indices)

# 从input中删除一个纬度
sample_indice = tf.squeeze(input=sample_indices, axis=1)
print(sample_indice)

tf.Tensor(
[[19]
 [42]
 [ 4]
 [ 6]
 [51]
 [52]
 [25]
 [36]
 [47]
 [41]
 [49]
 [52]
 [51]
 [48]
 [38]
 [27]
 [60]
 [11]
 [ 8]
 [ 4]
 [ 9]
 [42]
 [ 4]
 [18]
 [52]
 [51]
 [ 6]
 [ 9]
 [57]
 [57]
 [11]
 [49]
 [58]
 [59]
 [24]
 [27]
 [55]
 [15]
 [12]
 [16]
 [19]
 [ 4]
 [56]
 [13]
 [49]
 [39]
 [ 6]
 [29]
 [26]
 [ 0]
 [37]
 [16]
 [19]
 [60]
 [11]
 [57]
 [26]
 [11]
 [10]
 [ 9]
 [38]
 [36]
 [62]
 [17]
 [13]
 [ 8]
 [ 9]
 [30]
 [ 3]
 [54]
 [35]
 [38]
 [ 0]
 [44]
 [64]
 [57]
 [49]
 [ 6]
 [54]
 [53]
 [37]
 [ 9]
 [60]
 [36]
 [52]
 [ 8]
 [40]
 [52]
 [30]
 [ 6]
 [41]
 [46]
 [18]
 [43]
 [25]
 [32]
 [ 6]
 [12]
 [24]
 [34]], shape=(100, 1), dtype=int64)
tf.Tensor(
[19 42  4  6 51 52 25 36 47 41 49 52 51 48 38 27 60 11  8  4  9 42  4 18
 52 51  6  9 57 57 11 49 58 59 24 27 55 15 12 16 19  4 56 13 49 39  6 29
 26  0 37 16 19 60 11 57 26 11 10  9 38 36 62 17 13  8  9 30  3 54 35 38
  0 44 64 57 49  6 54 53 37  9 60 36 52  8 40 52 30  6 41 46 18 43 25 32
  6 12 24 34], shape=(100,), dtype=int64)


In [20]:
# 对比预测的数据
print("input:", repr("".join([idx2char[i] for i in np.array(input_example_batch[0])])))
print("output:", repr("".join([idx2char[i] for i in np.array(target_example_batch[0])])))
print("predict:", repr("".join([idx2char[i] for i in sample_indice])))

input: "on thee; and thou art too full\nOf the wars' surfeits, to go rove with one\nThat's yet unbruised: brin"
output: "n thee; and thou art too full\nOf the wars' surfeits, to go rove with one\nThat's yet unbruised: bring"
predict: 'Gd&,mnMXicknmjZOv;.&3d&Fnm,3ss;ktuLOqC?DG&rAka,QN\nYDGv;sN;:3ZXxEA.3R$pWZ\nfzsk,poY3vXn.bnR,chFeMT,?LV'


In [21]:
# from_logits=True：因为返回的是逻辑回归，所以设置TRUE
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(y_true=labels,
                                                     y_pred=logits,
                                                     from_logits=True,)

model.compile(optimizer="adam", loss=loss)
example_loss = loss(target_example_batch, example_batch_predictions)
print(example_loss.shape)
print(example_loss.numpy().mean())

(64, 100)
4.1744466


In [22]:
# 设置检查点
output_dir = "./out_generation_checkpoints"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
checkpoint_prefix = os.path.join(output_dir, 'ckpt_{epoch}')
# 保存权重信息
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True)

epochs = 10
history = model.fit(seq_dataset, epochs = epochs,
                    callbacks = [checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 535s 3s/step - loss: 2.5826
Epoch 2/10
172/172 [==============================] - 549s 3s/step - loss: 1.8956
Epoch 3/10
172/172 [==============================] - 549s 3s/step - loss: 1.6518
Epoch 4/10
172/172 [==============================] - 555s 3s/step - loss: 1.5154
Epoch 5/10
172/172 [==============================] - 656s 4s/step - loss: 1.4317
Epoch 6/10
172/172 [==============================] - 691s 4s/step - loss: 1.3748
Epoch 7/10
172/172 [==============================] - 646s 4s/step - loss: 1.3297
Epoch 8/10
172/172 [==============================] - 540s 3s/step - loss: 1.2906
Epoch 9/10
172/172 [==============================] - 541s 3s/step - loss: 1.2554
Epoch 10/10
172/172 [==============================] - 550s 3s/step - loss: 1.2216


In [23]:
# 使用训练好的模型
for input_example_batch, target_example_batch in seq_dataset.take(1):
    print(input_example_batch, input_example_batch.shape)
    print(target_example_batch, target_example_batch.shape)
    # 函数式调用
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape)
    
# 对比预测的数据
print("input:", repr("".join([idx2char[i] for i in np.array(input_example_batch[0])])))
print("output:", repr("".join([idx2char[i] for i in np.array(target_example_batch[0])])))
print("predict:", repr("".join([idx2char[i] for i in sample_indice])))

tf.Tensor(
[[57 46  1 ... 43  1 45]
 [ 1 57 59 ... 53 44  1]
 [52  1 42 ... 39 56 43]
 ...
 [39 54  1 ... 58 53  1]
 [47 41 43 ... 61 53 56]
 [56 53 61 ... 21  1 58]], shape=(64, 100), dtype=int32) (64, 100)
tf.Tensor(
[[46  1 46 ...  1 45 39]
 [57 59 47 ... 44  1 46]
 [ 1 42 53 ... 56 43 61]
 ...
 [54  1 58 ... 53  1 45]
 [41 43  8 ... 53 56 42]
 [53 61 52 ...  1 58 46]], shape=(64, 100), dtype=int32) (64, 100)
(64, 100, 65)
input: 'sh him our city,\nIn peril of precipitation\nFrom off the rock Tarpeian never more\nTo enter our Rome g'
output: 'h him our city,\nIn peril of precipitation\nFrom off the rock Tarpeian never more\nTo enter our Rome ga'
predict: 'Gd&,mnMXicknmjZOv;.&3d&Fnm,3ss;ktuLOqC?DG&rAka,QN\nYDGv;sN;:3ZXxEA.3R$pWZ\nfzsk,poY3vXn.bnR,chFeMT,?LV'


In [24]:
tf.train.latest_checkpoint(output_dir)

'./out_generation_checkpoints\\ckpt_10'

In [25]:
# 恢复最新的检查点
# 为保持此次预测步骤简单，将批大小设定为 1。
# 由于 RNN 状态从时间步传递到时间步的方式，模型建立好之后只接受固定的批大小。
# 若要使用不同的 batch_size 来运行模型，我们需要重建模型并从检查点中恢复权重。
model02 = build_model(vocab_size=vocab_size, embedding_dim=embedding_dim, batch_size=1, units=units)
model02.load_weights(tf.train.latest_checkpoint(output_dir))
model02.build(tf.TensorShape([1, None]))
model02.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [26]:
def generate_text(model, start_string):
    input_eval = [char2idx[c] for c in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    
    num_generate = 1000
    text_generate = []
    model.reset_states()
    
    for _ in range(num_generate):
        predictions = model(input_eval)
        
        predictions = tf.squeeze(predictions, 0)
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        
        #sample_indice = tf.random.categorical(logits=predictions[0], num_samples=1)
        #sample_indice = tf.squeeze(input=sample_indices, axis=1)
        text_generate.append(idx2char[predicted_id])
        input_eval = tf.expand_dims([predicted_id], 0)

    return start_string + "".join(text_generate)

In [27]:
print(generate_text(model02, "All:"))

All: mina pine.

Lord:
Welcome, you shall held it a man dot.

LEONTES:
That it struke and after me; counterbine song!
How now, he will go too cruel of mother.

CAMILLO:
My meerure find us mad: and I with
feeling matter gulls again. Prince! 
ISABELLA:
No, sir, your loviday, sir, interror inlk,
All my fortune shave before by yecreat me the succesf;
For Bariana A'll and Grumio as o'erwerr?

GONZALO:
'tis a good news, good fell with three honest present pears,' bossh,
And that borne war speak in peace and stim at him
And long i' the wash; and he used thyself.
My gaim, he'le wardly, I can.

FLORIZEL:
Fear not, Kine with you, thy purpose by me,
I hear me of my boots, bride alike:
And, as you shall faint.

PETRUCHIO:
Where, the acept unjust can do not rish snumbling
Should carry statutes him for pernipy; I
say, sir; as he, easy holds,
And bid you hear me over your husband!
Here be your name; but then supposess is nent;
And will recrieve twice, to my instructions upon you!

APHORSON:
Hord lord